# Preparing (Importing and initializing variables)

In [1]:
import os
import time
import pygame
import pyprind
import librosa
import numpy as np
import pandas as pd
import soundfile as sf
import sounddevice as sd

from playsound import playsound
from IPython.display import Audio 
from IPython.display import clear_output

from unified_algorithm_v2.unifiedalghorithm import unified_algorithm, unified_algorithm_mod

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model

from FTANet.network.ftanet_reverse import create_regularized_model as create_model
from FTANet.cfp import cfp_process
from FTANet.loader import batchize_test
from FTANet.evaluator import est

pygame 2.0.1 (SDL 2.0.14, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
# for midi player
midi_file = '.\database\MIR-QBSH\midiFile\\00001.mid'
freq = 44100    # audio CD quality
bitsize = -16   # unsigned 16 bit
channels = 2    # 1 is mono, 2 is stereo
buffer = 1024    # number of samples
pygame.mixer.init(freq, bitsize, channels, buffer)
pygame.mixer.music.set_volume(0.8) # optional volume 0 to 1.0

# for ftanet extraction
in_shape = (128, 320, 3)
model_path = 'E:\Kuliah\S2\Thesis\source code thesis\FTANet\model\\retrain\model_2007_r-epoch8.h5'
model = create_model(factor = 0.00001, rate = 0.5, input_shape=in_shape)
model.load_weights(model_path)

# for recording
samplerate = 48000  
duration = 8 # seconds
filename = 'demo.wav'


In [ ]:
def ftanet_extraction(model, query, batch_size = 128):
    preds = []
    data, CenFreq, time_arr = cfp_process(query, model_type='vocal', sr=8000, hop=80, window=768)
    feature = batchize_test(data, batch_size)
    
    for i in range(len(feature)):
        x = feature[i]
        
        # predict and concat
        num = x.shape[0] // batch_size
        if x.shape[0] % batch_size != 0:
            num += 1
        
        for j in range(num):
            # x: (batch_size, freq_bins, seg_len)
            if j == num - 1:
                X = x[j*batch_size : ]
                length = x.shape[0]-j*batch_size
            else:
                X = x[j*batch_size : (j+1)*batch_size]
                length = batch_size

            X = np.reshape(X, (1, 128, 320, 3))
            prediction = model.predict(X, length)
            preds.append(prediction)
            
    preds = np.concatenate(np.concatenate(preds, axis=0), axis=0)
    
    est_arr = est(preds, CenFreq, time_arr)
    f0 = []
    mel_time = []
    for x in est_arr:
        mel_time.append(x[0])
        f0.append(x[1])
    
    return mel_time, f0  

In [4]:
# def demo():
#     print("start recording...")
#     mydata = sd.rec(int(samplerate * duration), samplerate=samplerate,
#         channels=1, blocking=True)
#     print("end")
#     sd.wait()
#     sf.write(filename, mydata, samplerate)
    
#     # dont work in vscode
#     # play the recording
#     # samples, sample_rate = librosa.load(filename , sr = 16000)
#     # samples = librosa.resample(samples, sample_rate, 8000)
#     # Audio(samples,rate=8000, autoplay=True)
#     print("extracting melody...")
#     mel_time, f0 = ftanet_extraction(model, filename)
#     # print(mel_time, f0)


In [4]:
def play_music(music_file): # from https://gist.github.com/naotokui/29073690279056e9354e6259efbf8f30
    """
    stream music with mixer.music module in blocking manner
    this will stream the sound from disk while playing
    """
    clock = pygame.time.Clock()
    try:
        pygame.mixer.music.load(music_file)
        print ("Music file {} loaded!".format(music_file)) 
    except pygame.error:
        print ("File {} not found! ({})".format(music_file, pygame.get_error()))
        return
    pygame.mixer.music.play()
    while pygame.mixer.music.get_busy():
        # check if playback has finished
        clock.tick(30)

In [5]:
def demo_program():
    wav_list_path = 'wav_demo.txt'
    with open("wav_demo.txt",'r') as f:
        querylist = f.readlines()
    f.close()

    # bar = pyprind.ProgPercent(len(querylist), track_time=True, stream=1, title='Starting demo')
    for files in querylist:        
        files = files.replace("\n","")
        files = files.split(',')
        query = files[0]
        truth = files[1]
        
        print('Play the humming...')
        playsound(query)
        
        print('Extracting melody...') 
        mel_time, f0 = ftanet_extraction(model, query)
        
        print('Matching process...')
        result, alg, rank = unified_algorithm(f0, truth, compressed = 1)
        # print(result, alg, truth)
        # print(f0)
        result = list(result)[0]
        result = result.replace('midi_note', 'midiFile').replace('.csv', '.mid')
        # 
        try:
            play_music(result)
        except KeyboardInterrupt:
            # if user hits Ctrl/C then exit, (works only in console mode)    
            pygame.mixer.music.fadeout(1000)
            pygame.mixer.music.stop()
        clear_output(wait=True)

demo_program()

Play the humming...
Extracting melody...


In [ ]:
model.summary()